### Libraries

In [1]:
from typing import Union
import numpy as np
from qclib.gates.mcu import MCU
from qclib.gates.ldmcu import Ldmcu
from qiskit import (QuantumCircuit,
                    QuantumRegister,
                    transpile)
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.quantum_info import state_fidelity, Statevector
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import matplotlib.pyplot as plt

In [2]:
class Utilities:

    def __init__(self, operator, error):
        self.operator = operator
        self.error = error

    @staticmethod
    def pauli_matrices(string: str) -> Union[np.ndarray, None]:
        string_case = string.lower()
        match string_case:
            case "x":
                return np.array([
                    [0, 1],
                    [1, 0]
                ])
            case "y":
                return np.array([
                    [0, -1j],
                    [1j, 0]
                ])
            case "z":
                return np.array([
                    [1, 0],
                    [0, -1]
                ])
            case _:
                return None

    @staticmethod
    def transpose_conjugate(operator):
        return np.conjugate(np.transpose(operator))

    @staticmethod
    def pyramid_size(operator, error):
        mcu_dummy = MCU(operator, num_controls=100, error=error)
        # will be changed by mcu_dummy._get_n_base(x_dagger, error)
        return mcu_dummy._get_num_base_ctrl_qubits(operator, error)


In [3]:
def built_circuit(extra_qubits=0, pauli_string='x', error=0.1, approximated=True):
    
    pauli_matrix = Utilities.pauli_matrices(pauli_string)
    pauli_matrix_dagger = Utilities.transpose_conjugate(pauli_matrix)
    n_base = Utilities.pyramid_size(pauli_matrix_dagger, error)

    controls = QuantumRegister(n_base + extra_qubits, 'controls')
    target = QuantumRegister(1, 'target')
    circ = QuantumCircuit(controls, target)
    circ.x(list(range(len(controls))))
    
    if approximated:
        MCU.mcu(circ, pauli_matrix_dagger, controls, target, error)
    else:
        Ldmcu.ldmcu(circ, pauli_matrix_dagger, controls, target)
        
    # circ.measure_all()
    
    return circ

In [4]:
n_extra = 0
error = 0.1
matrix = 'x'

In [5]:
approx_circuit = built_circuit(n_extra, matrix, error)
# print(approx_circuit)

In [6]:
real_circuit = built_circuit(n_extra, matrix, error, False)
# print(real_circuit)

In [81]:
def state_vector(circuit):
    
    size = len(circuit)
    backend = GenericBackendV2(num_qubits=size)
    noise_model = NoiseModel.from_backend(backend)
    
    simulator = AerSimulator(noise_model=noise_model)
    transpiled_circuit = transpile(circuit, simulator, basis_gates= noise_model.basis_gates)
    transpiled_circuit.save_statevector()
    
    job = simulator.run(transpiled_circuit)
    result = job.result()
    sv = result.get_statevector()
    # result = job.result().get_counts(0)
    # d_m = DensityMatrix(transpiled_circuit)
    return sv, size

In [33]:
def density_matrix(circuit):
    
    size = len(circuit)
    backend = GenericBackendV2(num_qubits=size, seed=42)
    noise_model = NoiseModel.from_backend(backend)
    
    simulator = AerSimulator(noise_model=noise_model, method='density_matrix')
    transpiled_circuit = transpile(circuit, simulator, basis_gates= noise_model.basis_gates)
    transpiled_circuit.save_density_matrix()
    #d_m = transpiled_circuit.density_matrix()
    #return d_m, size
    
    job = simulator.run(transpiled_circuit)
    result = job.result()
    d_m = result.data(0)
    # result = job.result().get_counts(0)
    # d_m = DensityMatrix(transpiled_circuit)
    return d_m, size

In [82]:
def state_vector_noiseless(circuit):
    
    size = len(circuit)
    backend = GenericBackendV2(num_qubits=size)
    
    simulator = AerSimulator()
    transpiled_circuit = transpile(circuit, simulator)
    transpiled_circuit.save_statevector()
    
    job = simulator.run(transpiled_circuit)
    result = job.result()
    sv = result.get_statevector()
    # result = job.result().get_counts(0)
    # d_m = DensityMatrix(transpiled_circuit)
    return sv, size

### Noisy

In [39]:
exact_decomposition_dm, n1 = density_matrix(real_circuit)
print(exact_decomposition_dm)

{'density_matrix': DensityMatrix([[ 1.06498848e-03-8.63804054e-19j,
                 0.00000000e+00+0.00000000e+00j,
                 2.31342167e-06-1.92718703e-07j, ...,
                 0.00000000e+00+0.00000000e+00j,
                 4.37801671e-06+4.91313405e-07j,
                 0.00000000e+00+0.00000000e+00j],
               [ 0.00000000e+00+0.00000000e+00j,
                 5.13097261e-04+1.20046507e-19j,
                 0.00000000e+00+0.00000000e+00j, ...,
                 1.69213272e-05+3.57807018e-06j,
                 0.00000000e+00+0.00000000e+00j,
                -1.28664056e-07+3.07416464e-06j],
               [ 2.31342167e-06+1.92718703e-07j,
                 0.00000000e+00+0.00000000e+00j,
                 2.60853494e-04+9.27552899e-20j, ...,
                 0.00000000e+00+0.00000000e+00j,
                 2.08138353e-05+2.24585688e-08j,
                 0.00000000e+00+0.00000000e+00j],
               ...,
               [ 0.00000000e+00+0.00000000e+00j,
            

In [15]:

t_array = np.zeros(2**n1)
t_array[-1] = 1
t_state_vector = Statevector(t_array)
t_dm = DensityMatrix()
approximated_decomposition_dm, n2 = density_matrix(approx_circuit)
print(state_fidelity(t_state_vector, exact_decomposition_dm))
print(state_fidelity(t_state_vector, approximated_decomposition_dm))

AttributeError: 'QuantumCircuit' object has no attribute 'get_density_matrix'

In [84]:
exact_decomposition_s_vector, n1 = state_vector(real_circuit)
t_array = np.zeros(2**n1)
t_array[-1] = 1
t_state_vector = Statevector(t_array)
approximated_decomposition_s_vector, n2 = state_vector(approx_circuit)
print(state_fidelity(t_state_vector, exact_decomposition_s_vector))
print(state_fidelity(t_state_vector, approximated_decomposition_s_vector))

0.9998373116472037
2.5299227111982178e-27


### Noiseless

In [51]:
exact_decomposition_s_vector, n1 = state_vector_noiseless(real_circuit)
t_array = np.zeros(2**n1)
t_array[-1] = 1
t_state_vector = Statevector(t_array)
approximated_decomposition_s_vector, n2 = state_vector_noiseless(approx_circuit)
print(state_fidelity(t_state_vector, exact_decomposition_s_vector))
print(state_fidelity(t_state_vector, approximated_decomposition_s_vector))

1.000000000000004
0.9975923633361053
